In [1]:
import torchvision.models as models
import torch
import torch.nn as nn

# 加載 GoogLeNet 模型
model = models.googlenet(pretrained=True)
print(model)

input_shape = (3, 224, 224)

/opt/conda/envs/tensorflow/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/envs/tensorflow/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /home/Hank/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:07<00:00, 7.44MB/s]

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [2]:
# Calculating the total number of parameters in the model
total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters: ", total_params)

Total number of parameters:  6624904


In [3]:
# Calculating the size of the model's parameters in bytes
param_size = sum(p.numel() * p.element_size() for p in model.parameters())
print("Total memory for parameters: ", param_size)

Total memory for parameters:  26499616


In [4]:
import torchinfo
torchinfo.summary(model, (3, 224, 224), batch_dim=0, col_names=("input_size", "output_size", "num_params", "kernel_size", "mult_adds"), verbose=0)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
GoogLeNet                                [1, 3, 224, 224]          [1, 1000]                 --                        --                        --
├─BasicConv2d: 1-1                       [1, 3, 224, 224]          [1, 64, 112, 112]         --                        --                        --
│    └─Conv2d: 2-1                       [1, 3, 224, 224]          [1, 64, 112, 112]         9,408                     [7, 7]                    118,013,952
│    └─BatchNorm2d: 2-2                  [1, 64, 112, 112]         [1, 64, 112, 112]         128                       --                        128
├─MaxPool2d: 1-2                         [1, 64, 112, 112]         [1, 64, 56, 56]           --                        3                         --
├─BasicConv2d: 1-3                       [1, 64, 56, 56]           [1, 64, 56, 56]           --

### Caculate Computation Requirement

In [4]:
def calculate_output_shape(input_shape, layer):
    # Calculate the output shape for Conv2d, MaxPool2d, and Linear layers
    if isinstance(layer, (nn.Conv2d, nn.MaxPool2d)):
        kernel_size = (
            layer.kernel_size
            if isinstance(layer.kernel_size, tuple)
            else (layer.kernel_size, layer.kernel_size)
        )
        stride = (
            layer.stride
            if isinstance(layer.stride, tuple)
            else (layer.stride, layer.stride)
        )
        padding = (
            layer.padding
            if isinstance(layer.padding, tuple)
            else (layer.padding, layer.padding)
        )
        dilation = (
            layer.dilation
            if isinstance(layer.dilation, tuple)
            else (layer.dilation, layer.dilation)
        )

        output_height = (
            input_shape[1] + 2 * padding[0] - dilation[0] * (kernel_size[0] - 1) - 1
        ) // stride[0] + 1
        output_width = (
            input_shape[2] + 2 * padding[1] - dilation[1] * (kernel_size[1] - 1) - 1
        ) // stride[1] + 1
        return (
            layer.out_channels if hasattr(layer, "out_channels") else input_shape[0],
            output_height,
            output_width,
        )
    elif isinstance(layer, nn.Linear):
        # For Linear layers, the output shape is simply the layer's output features
        return (layer.out_features,)
    else:
        return input_shape


def calculate_macs(layer, input_shape, output_shape):
    # Calculate MACs for Conv2d and Linear layers
    if isinstance(layer, nn.Conv2d):
        kernel_ops = (
            layer.kernel_size[0]
            * layer.kernel_size[1]
            * (layer.in_channels / layer.groups)
        )
        output_elements = output_shape[1] * output_shape[2]
        macs = int(kernel_ops * output_elements * layer.out_channels)
        return macs
    elif isinstance(layer, nn.Linear):
        # For Linear layers, MACs are the product of input features and output features
        macs = int(layer.in_features * layer.out_features)
        return macs
    else:
        return 0

In [5]:
# Initial input shape
input_shape = (3, 224, 224)
total_macs = 0

# Iterate through the layers of the model
for name, layer in model.named_modules():
    if isinstance(layer, (nn.Conv2d, nn.Linear)):
        output_shape = calculate_output_shape(input_shape, layer)
        macs = calculate_macs(layer, input_shape, output_shape)
        total_macs += macs
        if isinstance(layer, (nn.Conv2d, nn.Linear)):
            print(
                f"Layer: {name}, Type: {type(layer).__name__}, Input Shape: {input_shape}, Output Shape: {output_shape}, MACs: {macs}"
            )
        input_shape = output_shape  # Update the input shape for the next layer

print(f"Total MACs: {total_macs}")

Layer: conv1.conv, Type: Conv2d, Input Shape: (3, 224, 224), Output Shape: (64, 112, 112), MACs: 118013952
Layer: conv2.conv, Type: Conv2d, Input Shape: (64, 112, 112), Output Shape: (64, 112, 112), MACs: 51380224
Layer: conv3.conv, Type: Conv2d, Input Shape: (64, 112, 112), Output Shape: (192, 112, 112), MACs: 1387266048
Layer: inception3a.branch1.conv, Type: Conv2d, Input Shape: (192, 112, 112), Output Shape: (64, 112, 112), MACs: 154140672
Layer: inception3a.branch2.0.conv, Type: Conv2d, Input Shape: (64, 112, 112), Output Shape: (96, 112, 112), MACs: 231211008
Layer: inception3a.branch2.1.conv, Type: Conv2d, Input Shape: (96, 112, 112), Output Shape: (128, 112, 112), MACs: 1387266048
Layer: inception3a.branch3.0.conv, Type: Conv2d, Input Shape: (128, 112, 112), Output Shape: (16, 112, 112), MACs: 38535168
Layer: inception3a.branch3.1.conv, Type: Conv2d, Input Shape: (16, 112, 112), Output Shape: (32, 112, 112), MACs: 57802752
Layer: inception3a.branch4.1.conv, Type: Conv2d, Input S

### Use forward hooks to extract the output activations of the Conv2d layers

In [8]:
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook


# Dictionary to store activations from each layer
activation = {}

# Register hook to each linear layer
for layer_name, layer in model.named_modules():
    if isinstance(layer, torch.nn.Conv2d):
        # Register forward hook
        layer.register_forward_hook(get_activation(layer_name))

# Run model inference
data = torch.randn(1, 3, 224, 224)
output = model(data)

# Access the saved activations
for layer in activation:
    print(f"Activation from layer {layer}: {activation[layer].shape}")

Activation from layer conv1.conv: torch.Size([1, 64, 112, 112])
Activation from layer conv2.conv: torch.Size([1, 64, 56, 56])
Activation from layer conv3.conv: torch.Size([1, 192, 56, 56])
Activation from layer inception3a.branch1.conv: torch.Size([1, 64, 28, 28])
Activation from layer inception3a.branch2.0.conv: torch.Size([1, 96, 28, 28])
Activation from layer inception3a.branch2.1.conv: torch.Size([1, 128, 28, 28])
Activation from layer inception3a.branch3.0.conv: torch.Size([1, 16, 28, 28])
Activation from layer inception3a.branch3.1.conv: torch.Size([1, 32, 28, 28])
Activation from layer inception3a.branch4.1.conv: torch.Size([1, 32, 28, 28])
Activation from layer inception3b.branch1.conv: torch.Size([1, 128, 28, 28])
Activation from layer inception3b.branch2.0.conv: torch.Size([1, 128, 28, 28])
Activation from layer inception3b.branch2.1.conv: torch.Size([1, 192, 28, 28])
Activation from layer inception3b.branch3.0.conv: torch.Size([1, 32, 28, 28])
Activation from layer inception